In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd

In [3]:
# Load our data.
df = pd.read_csv("data/Purchase_Card_Transactions.csv")
df = df.dropna()

In [5]:
index_names = df[df['TRANSACTION_AMOUNT'] <= 0 ].index 
# drop these row indexes
# from dataFrame 
df.drop(index_names, inplace = True) 

In [6]:
# Clean the vendor name.
# Get rid of all characters except for [a-Z]
df['VENDOR_NAME_CLEAN'] = df['VENDOR_NAME'].str.replace(r'\s+', '_').str.replace(r'\W+', '').str.replace(r'\d+', '')

In [7]:
# Sort vendors and agencies alphabetically.
list_vendors = sorted(df['VENDOR_NAME_CLEAN'].unique().astype(str))
list_agencies = sorted(df['AGENCY'].unique().astype(str))

In [8]:
# Create mapping from vendor and agency names to indices.
list_vendors_map = {list_vendors[i]: i for i in range(len(list_vendors))}
list_agencies_map = {list_agencies[i]: i for i in range(len(list_agencies))}

In [9]:
# Create indices.
df['VENDOR_NAME_NUM'] = df['VENDOR_NAME_CLEAN'].apply(lambda i: list_vendors_map[i])
df['AGENCY_NUM'] = df['AGENCY'].apply(lambda i: list_agencies_map[i])

In [10]:
# Create new dataframe.
newdf = df[['AGENCY_NUM', 'VENDOR_NAME_NUM', 'TRANSACTION_AMOUNT', 'TRANSACTION_DATE', 'VENDOR_STATE_PROVINCE']].copy(deep = True)

In [11]:
# Save the dataframe.
newdf.to_csv("data/Cleaned_Purchase_Card_Transactions.csv")